There are four main types of prompts that we can use for prompting.
1. Using a <b>RolePrompt</b> Template - Defines the expectation from the model in a "role-playing" mode and provides the placeholders for user input.
2. Using a <b>Few shot Prompt</b> Template - give examples to the model in terms of what is expected.
3. Using <b>Chain Prompting</b> Templates - Provide a set of sequential prompts where the response of prompt 1, for example, is the input for prompt 2
4. Using <b>Chain of Reasoning</b> Templates - Encourage the model to provide step-by-step description of its "thought" process

## RolePrompt Template

Provide a role to the model to help it finetune its response

In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

D:\miniconda3\envs\langchain\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.10) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


#### Create the role and define what the user inputs will be

In [2]:
role='You are a futuristic song writer who specializes in generating creative song titles. I need your help in creating a song title about {theme} in the genre {genre} for the year {year}'

In [3]:
inputs=['theme','genre','year'] #These are the three input variables that the user will give at every interaction

#### Create the Prompt Template

In [4]:
prompt=PromptTemplate(template=role, input_variables=inputs)

In [5]:
### Example of how the prompt will look when it is submitted to the model
prompt.format_prompt(theme='starry skies',genre='electronica',year='2030')


StringPromptValue(text='You are a futuristic song writer who specializes in generating creative song titles. I need your help in creating a song title about starry skies in the genre electronica for the year 2030')

#### Create the model

In [6]:
llm = OpenAI(model='text-davinci-003',temperature=0)

#### Create a chain

In [7]:
llmchain=LLMChain(llm=llm, prompt=prompt,verbose=True)

#### Run the chain

In [13]:
response=llmchain.run(theme='night sky',genre='electronica',year='2030')
print(response)



> Entering new  chain...
Prompt after formatting:
You are a futuristic song writer who specializes in generating creative song titles. I need your help in creating a song title about night sky in the genre electronica for the year 2030

> Finished chain.
.

"Starry Nights 2030"


In [14]:
response=llmchain.run(theme='starry sky',genre='electronica',year='2030')
print(response)



> Entering new  chain...
Prompt after formatting:
You are a futuristic song writer who specializes in generating creative song titles. I need your help in creating a song title about starry sky in the genre electronica for the year 2030

> Finished chain.
.

"Starry Skies 2030: An Electronica Odyssey"


In [15]:
response=llmchain.run(theme='starry sky',genre='rock',year='2030')
print(response)



> Entering new  chain...
Prompt after formatting:
You are a futuristic song writer who specializes in generating creative song titles. I need your help in creating a song title about starry sky in the genre rock for the year 2030

> Finished chain.
.

"Starry Skies of 2030"


In [16]:
response=llmchain.run(theme='interstellar travel',genre='rock',year='2030')
print(response)



> Entering new  chain...
Prompt after formatting:
You are a futuristic song writer who specializes in generating creative song titles. I need your help in creating a song title about interstellar travel in the genre rock for the year 2030

> Finished chain.
.

"Interstellar Odyssey: A Rock Anthem for 2030"


In [17]:
response=llmchain.run(theme='interstellar travel',genre='rap',year='2030')
print(response)



> Entering new  chain...
Prompt after formatting:
You are a futuristic song writer who specializes in generating creative song titles. I need your help in creating a song title about interstellar travel in the genre rap for the year 2030

> Finished chain.
.

"Interstellar Hustle: A Rap Odyssey of the Future"


<b>The above titles while relevant sound incredibly boring. Maybe, if we try increasing the temperature of the LLM? The temperature metric allows the model to be more creative and diverge away from the prompt while anchoring itself to it.</b>

#### Changing the temperature parameter of the model

In [19]:
llm = OpenAI(model='text-davinci-003',temperature=0.9)
llmchain=LLMChain(llm=llm, prompt=prompt,verbose=True)

In [21]:
response=llmchain.run(theme='interstellar travel',genre='rock',year='2030')
print(response)



> Entering new  chain...
Prompt after formatting:
You are a futuristic song writer who specializes in generating creative song titles. I need your help in creating a song title about interstellar travel in the genre rock for the year 2030

> Finished chain.
.

"Moon Light Drive 2050"


In [22]:
response=llmchain.run(theme='interstellar travel',genre='rap',year='2030')
print(response)



> Entering new  chain...
Prompt after formatting:
You are a futuristic song writer who specializes in generating creative song titles. I need your help in creating a song title about interstellar travel in the genre rap for the year 2030

> Finished chain.
.

"Journey Through the Starlight: A Rap about Interstellar Travel in 2030"


<b>The names actually sound far more album-ish and creative

## Few Shot Prompt Template

In [25]:
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts import PromptTemplate

The FewShotPrompt structure looks a bit like this:<br>
<ul>
    <li>< prompt > The structure of the input data. This structure should be similar to the examples provided</li>
    <li>< prefix for the examples> Tell the model that we want outputs that are similar to the examples given</li>
    <li>< examples > The actual examples</li>
    <li>< suffix for the examples > Give the format that we want the output in</li>
</ul>

#### Create the prompt

In [27]:
template="""
Color: {color}
Emotion: {emotion}\n
"""
inputs=['color','emotion']

In [28]:
prompt=PromptTemplate(input_variables=inputs, template=template)

#### Create the prefix

In [29]:
prefix='Here are some examples of colors and the emotions associated with them\n\n'

#### Create the examples

In [30]:
#Each example needs to be a dictionary in the same format as the template
examples=[
    {"color":"red", "emotion":"anger"},
    {"color":"blue" , "emotion":"serenity"},
    {"color":"green","emotion":"tranquility"},
]

#### Create the suffix

In [47]:
suffix= "Given a new color, use the examples provided to identify the emotion for the color {color} in the format:\n\nColor: {color}\nEmotion:"

#### Create the FewShotPromptTemplate

In [48]:
fspt=FewShotPromptTemplate(example_prompt=prompt,prefix=prefix,suffix=suffix,examples=examples,input_variables=['color'],example_separator='\n')

#### What does the final prompt look like?

In [49]:
print(fspt.format_prompt(color='purple'))

text='Here are some examples of colors and the emotions associated with them\n\n\n\nColor: red\nEmotion: anger\n\n\n\nColor: blue\nEmotion: serenity\n\n\n\nColor: green\nEmotion: tranquility\n\n\nGiven a new color, use the examples provided to identify the emotion for the color purple in the format:\n\nColor: purple\nEmotion:'


#### Create the model

In [50]:
llm=OpenAI(model='text-davinci-003', temperature=0.9)

#### Create the chain

In [51]:
llmchain=LLMChain(llm=llm, prompt=fspt)

#### Run the chain

In [52]:
response=llmchain.run(color='purple')
print(response)

 calmness


#### What if we change the temperature to 0

In [65]:
llm=OpenAI(model='text-davinci-003', temperature=0)
llmchain=LLMChain(llm=llm, prompt=fspt)

In [66]:
response=llmchain.run(color='purple')
print(response)

 calmness


<b> The response does not seem to change much based on the temperature. However, calmness seems to be an average of anger and serenity, the same way that purple can be construed as the average of red and blue

In [67]:
print('Color: yellow\tEmotion:',llmchain.run(color='yellow'))
print('Color: orange\tEmotion:',llmchain.run(color='orange'))
print('Color: Brown\tEmotion:',llmchain.run(color='Brown'))

Color: yellow	Emotion:  optimism
Color: orange	Emotion:  optimism
Color: Brown	Emotion:  warmth


In [68]:
llm=OpenAI(model='text-davinci-003', temperature=0.9)
llmchain=LLMChain(llm=llm, prompt=fspt)
print('Color: yellow\tEmotion:',llmchain.run(color='yellow'))
print('Color: orange\tEmotion:',llmchain.run(color='orange'))
print('Color: Brown\tEmotion:',llmchain.run(color='Brown'))

Color: yellow	Emotion:  optimism
Color: orange	Emotion:  optimism
Color: Brown	Emotion:  warmth


So yellow and orange seem to be strongly correlated. <br>
<b>However, the exercise does highlight the slight instability of the model's word generation prowess as shown below

In [70]:
llm=OpenAI(model='text-davinci-003', temperature=0.9)
llmchain=LLMChain(llm=llm, prompt=fspt)
print('Color: yellow\tEmotion:',llmchain.run(color='yellow'))
print('Color: orange\tEmotion:',llmchain.run(color='orange'))
print('Color: Brown\tEmotion:',llmchain.run(color='Brown'))

Color: yellow	Emotion:  optimism
Color: orange	Emotion:  optimism
Color: Brown	Emotion:  stability


All we did was re-perform the exercise with no change between the previous two cells. However the emotion of <b>brown</b> changed from <b>warmth</b> to <b>stability</b>. 

#### Change temperature to 0

In [72]:
llm=OpenAI(model='text-davinci-003', temperature=0)
llmchain=LLMChain(llm=llm, prompt=fspt)
print('Color: yellow\tEmotion:',llmchain.run(color='yellow'))
print('Color: orange\tEmotion:',llmchain.run(color='orange'))
print('Color: Brown\tEmotion:',llmchain.run(color='Brown'))

Color: yellow	Emotion:  optimism
Color: orange	Emotion:  optimism
Color: Brown	Emotion:  warmth


In [75]:
llm=OpenAI(model='text-davinci-003', temperature=0)
llmchain=LLMChain(llm=llm, prompt=fspt)
print('Color: yellow\tEmotion:',llmchain.run(color='yellow'))
print('Color: orange\tEmotion:',llmchain.run(color='orange'))
print('Color: Brown\tEmotion:',llmchain.run(color='Brown'))

Color: yellow	Emotion:  optimism
Color: orange	Emotion:  optimism
Color: Brown	Emotion:  warmth


So keeping the temperature = 0 not only ensures adherence to the prompt but also increases the stability of the response over multiple interactions

## ChainPrompting

In [107]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

#### Create two prompts to be chained

In [108]:
template1='What is the name of the scientist who was responsible for the raman effect?'
prompt1=PromptTemplate(
    template=template1,
    input_variables=[])

In [109]:
template2='What were the other discoveries attributed to {scientist}?'
prompt2=PromptTemplate(
template=template2,
input_variables=['scientist'])

#### Create the model

In [110]:
llm=OpenAI(model='text-davinci-003',temperature=0)

#### Create the chain and run it for the first prompt

In [111]:
llmchain1=LLMChain(prompt=prompt1,llm=llm)
response=llmchain1.run({})

In [112]:
print(response.strip())

The scientist responsible for the Raman effect is Indian physicist Sir Chandrasekhara Venkata Raman.


Getting the entire sentence as the answer instead of just the name won't act well as the input data for our second prompt.<br> <b> A hack to get only the name is to structure the prompt as follows

In [113]:
template1a="""What is the name of the scientist who was responsible for the raman effect? Answer: """
prompt1a=PromptTemplate(
    template=template1a,
    input_variables=[])

In [114]:
llmchain1=LLMChain(prompt=prompt1a,llm=llm)
response1=llmchain1.run({})
print(response1.strip())

Sir Chandrasekhara Venkata Raman (C.V. Raman)


#### Chaining the second prompt

In [118]:
llmchain2=LLMChain(prompt=prompt2, llm=llm)

In [123]:
response2=llmchain2(response1.strip())

In [124]:
print(response2)

{'scientist': 'Sir Chandrasekhara Venkata Raman (C.V. Raman)', 'text': '\n\nIn addition to his Nobel Prize-winning discovery of the Raman Effect, C.V. Raman made several other important contributions to the field of physics. These include the discovery of the Raman-Nath diffraction, the Raman-Nath theory of diffraction, the Raman-Kashyap effect, the Raman-Kashyap theory of diffraction, the Raman-Kashyap-Raman effect, the Raman-Kashyap-Raman theory of diffraction, the Raman-Kashyap-Raman-Nath effect, the Raman-Kashyap-Raman-Nath theory of diffraction, the Raman-Kashyap-Raman-Nath-Raman effect, the Raman-Kashyap-Raman-Nath-Raman theory of diffraction, the Raman-Kashyap-Raman-Nath-Raman-Kashyap effect, and the Raman-Kashyap-Raman-Nath-Raman-Kashyap theory of diffraction. He also made important contributions to the fields of optics, acoustics, and spect'}


In [131]:
print(response2['scientist'],'\n',response2['text'])

Sir Chandrasekhara Venkata Raman (C.V. Raman) 
 

In addition to his Nobel Prize-winning discovery of the Raman Effect, C.V. Raman made several other important contributions to the field of physics. These include the discovery of the Raman-Nath diffraction, the Raman-Nath theory of diffraction, the Raman-Kashyap effect, the Raman-Kashyap theory of diffraction, the Raman-Kashyap-Raman effect, the Raman-Kashyap-Raman theory of diffraction, the Raman-Kashyap-Raman-Nath effect, the Raman-Kashyap-Raman-Nath theory of diffraction, the Raman-Kashyap-Raman-Nath-Raman effect, the Raman-Kashyap-Raman-Nath-Raman theory of diffraction, the Raman-Kashyap-Raman-Nath-Raman-Kashyap effect, and the Raman-Kashyap-Raman-Nath-Raman-Kashyap theory of diffraction. He also made important contributions to the fields of optics, acoustics, and spect


#### Trying to chain the second prompt with 'response' (The full sentence answer)

In [126]:
response3=llmchain2(response.strip())
print(response3)

{'scientist': 'The scientist responsible for the Raman effect is Indian physicist Sir Chandrasekhara Venkata Raman.', 'text': '\n\nOther discoveries attributed to Sir Chandrasekhara Venkata Raman include:\n\n1. The Raman-Nath theory of diffraction of light\n2. The Raman-Nath effect\n3. The Raman-Sommerfeld theory of diffraction\n4. The Raman-Nath effect in X-rays\n5. The Raman-Nath effect in electron diffraction\n6. The Raman-Nath effect in neutron diffraction\n7. The Raman-Nath effect in acoustic waves\n8. The Raman-Nath effect in surface waves\n9. The Raman-Nath effect in optical fibers\n10. The Raman-Nath effect in semiconductors\n11. The Raman-Nath effect in superconductors\n12. The Raman-Nath effect in plasmas\n13. The Raman-Nath effect in liquid crystals\n14. The Raman-Nath effect in polymers\n15. The Raman-Nath effect in nanostructures\n16. The Raman-Nath effect in nanomaterials\n17. The Raman-Nath effect in nanoelectronics\n18.'}


In [130]:
print(response3['scientist'],'\n',response3['text'])

The scientist responsible for the Raman effect is Indian physicist Sir Chandrasekhara Venkata Raman. 
 

Other discoveries attributed to Sir Chandrasekhara Venkata Raman include:

1. The Raman-Nath theory of diffraction of light
2. The Raman-Nath effect
3. The Raman-Sommerfeld theory of diffraction
4. The Raman-Nath effect in X-rays
5. The Raman-Nath effect in electron diffraction
6. The Raman-Nath effect in neutron diffraction
7. The Raman-Nath effect in acoustic waves
8. The Raman-Nath effect in surface waves
9. The Raman-Nath effect in optical fibers
10. The Raman-Nath effect in semiconductors
11. The Raman-Nath effect in superconductors
12. The Raman-Nath effect in plasmas
13. The Raman-Nath effect in liquid crystals
14. The Raman-Nath effect in polymers
15. The Raman-Nath effect in nanostructures
16. The Raman-Nath effect in nanomaterials
17. The Raman-Nath effect in nanoelectronics
18.


<b>Wow! Two different input structures but it still works! Ironically, the second prompt is better formatted than the first !

## Chain of Thought  Prompting

Chain of Thought Prompting (CoT) is a technique developed to encourage large language models to explain their reasoning process, <a href="https://arxiv.org/abs/2201.11903">leading to more accurate results</a>.<br> Chain of Thought template can be designed by simply putting together for e.g. a chain of few shot prompt templates <br>
However, while sharing the examples, e.g. {q: < enter the question >, a: < enter the answer >}, the value of the key 'a' <b>should not just provide the answer but also the steps taken to arrive at the answer</b>.

In [136]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.llms import OpenAI

### Using only prompts without FewShotPromptTemplate

#### Create the prompt

In [139]:
template='I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?\nAnswer:'
prompt=PromptTemplate(template=template, input_variables=[])

#### Create the model

In [140]:
llm=OpenAI(model='text-davinci-003', temperature=0)

#### Run the model

In [143]:
prompt

PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?\nAnswer:', template_format='f-string', validate_template=True)

In [144]:
response=llm(prompt=prompt.template)

In [145]:
print(response)

 12 apples


<b><font color= 'red'>&#10006;</font></b><b> This is actually wrong! 10-2-2+5-1 = 10 apples !</b>

#### Now trying again with the step by step prompting

In [148]:
template1='Question: I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?\nAnswer: Let us think step by step'
prompt1=PromptTemplate(template=template1, input_variables=[])

In [149]:
response1=llm(prompt=prompt1.template)

In [150]:
print(response1)

.

Initially, you had 10 apples.
You gave 2 apples to the neighbor and 2 to the repairman.
So, you had 10 - 2 - 2 = 6 apples left.
You then bought 5 more apples.
So, you had 6 + 5 = 11 apples.
Finally, you ate 1 apple.
So, you had 11 - 1 = 10 apples left.

Therefore, you remained with 10 apples.


<b>Welcome to the right answer. The only difference was the way we prompted the model to think step by step !</b>

### Chain of thought reasoning using FewShotPrompt

In [178]:
example_template = """
User: {question}
AI: {answer}
"""

In [210]:
example_set_1=[
    {'question':'I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?',
     'answer':'10 apples'},
    {'question':'While shopping for music online, Zoe bought 3 country albums and 5 pop albums. Each album came with a lyrics sheet and had 3 songs. How many songs did Zoe buy in total?',
    'answer':'24 songs'}
]

In [180]:
prompt_template1=PromptTemplate(template=example_template,input_variables=['question','answer'])

In [194]:
prefix="The following are excerpts from conversations with an AI math coach. The assistant provides answers to the users' questions. Here are some examples: "

In [203]:
suffix='''user:{question}
AI:'''

In [213]:
fspt_math=FewShotPromptTemplate(prefix=prefix,examples=example_set_1, example_prompt=prompt_template1,suffix=suffix, input_variables=['question'],example_separator="\n\n")

In [214]:
print(fspt_math)

input_variables=['question'] output_parser=None partial_variables={} examples=[{'question': 'I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?', 'answer': '10 apples'}, {'question': 'While shopping for music online, Zoe bought 3 country albums and 5 pop albums. Each album came with a lyrics sheet and had 3 songs. How many songs did Zoe buy in total?', 'answer': '24 songs'}] example_selector=None example_prompt=PromptTemplate(input_variables=['question', 'answer'], output_parser=None, partial_variables={}, template='\nUser: {question}\nAI: {answer}\n', template_format='f-string', validate_template=True) suffix='user:{question}\nAI:' example_separator='\n\n' prefix="The following are excerpts from conversations with an AI math coach. The assistant provides answers to the users' questions. Here are some examples: " template_format='f-string' validate_template

In [215]:
llmchain_math=LLMChain(llm=llm, prompt=fspt_math,verbose=True)

In [216]:
question1='Roger has 5 tennis balls. He buys two more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?'

In [217]:
response_math=llmchain_math.run(question1)



> Entering new  chain...
Prompt after formatting:
The following are excerpts from conversations with an AI math coach. The assistant provides answers to the users' questions. Here are some examples: 


User: I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?
AI: 10 apples



User: While shopping for music online, Zoe bought 3 country albums and 5 pop albums. Each album came with a lyrics sheet and had 3 songs. How many songs did Zoe buy in total?
AI: 24 songs


user:Roger has 5 tennis balls. He buys two more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
AI:

> Finished chain.


In [218]:
print(response_math)

 11 tennis balls


<b><font color= 'green'>&#10004;</font></b> This is the right answer. We did not chain of reasoning for this question

In [219]:
question2='A juggler can juggle 16 balls. Half the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?'

In [220]:
response_math2=llmchain_math.run(question2)



> Entering new  chain...
Prompt after formatting:
The following are excerpts from conversations with an AI math coach. The assistant provides answers to the users' questions. Here are some examples: 


User: I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?
AI: 10 apples



User: While shopping for music online, Zoe bought 3 country albums and 5 pop albums. Each album came with a lyrics sheet and had 3 songs. How many songs did Zoe buy in total?
AI: 24 songs


user:A juggler can juggle 16 balls. Half the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?
AI:

> Finished chain.


In [221]:
print(response_math2)

 8 blue golf balls


<b><font color= 'red'>&#10006;</font></b> This the wrong answer. The correct answer is 0.5 * 0.5 * 16 = 4 blue balls

<b><font color= 'green'>&#10004;</font></b><br> <b><font color= 'red'>&#10006;</font></b>

#### Solving question2 with chain of thought reasoning

In [235]:
reason_template='''Please provide your step-by-step reasoning. User: {question}
AI: Let's think step by step {answer}'''
reason_prompt=PromptTemplate(input_variables=['question','answer'],template=reason_template)

In [ ]:
prefix="The following are excerpts from conversations with an AI math coach. The assistant provides answers to the users' questions. Here are some examples: "

In [241]:
example_set_2=[
    {'question':'I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?',
     'answer':'''Let's think step by step.First, you started with 10 apples. You gave away 2 apples to the neighbor and 2 to the repairman, so you had 6 apples left.
    Then you bought 5 more apples, so now you had 11 apples. Finally, you ate 1 apple, so you would remain with 10 apples.'''},
    {'question':'While shopping for music online, Zoe bought 3 country albums and 5 pop albums. Each album came with a lyrics sheet and had 3 songs. How many songs did Zoe buy in total?',
    'answer':"Let's think step by step. Zoe bought 3 country albums. Each album has 3 songs. So she bought 3*3 = 9 songs from the country albums. Zoe bought 5 pop albums. Each album has 3 songs. So she bought 5*3 = 15 songs from the pop album. Zoe bought 9+15=24 songs in total. The answer is 24"}
]

In [242]:
fspt_reason=FewShotPromptTemplate(
    prefix=prefix, 
    examples=example_set_2, 
    example_prompt=reason_prompt, 
    suffix=suffix,
    example_separator='\n\n',
    input_variables=['question'])

In [243]:
llmchain_reason=LLMChain(prompt=fspt_reason, llm=llm, verbose=True)

In [244]:
response_reason=llmchain_reason.run(question2)



> Entering new  chain...
Prompt after formatting:
The following are excerpts from conversations with an AI math coach. The assistant provides answers to the users' questions. Here are some examples: 

Please provide your step-by-step reasoning. User: I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?
AI: Let us think step by step Let's think step by step.First, you started with 10 apples. You gave away 2 apples to the neighbor and 2 to the repairman, so you had 6 apples left.
    Then you bought 5 more apples, so now you had 11 apples. Finally, you ate 1 apple, so you would remain with 10 apples.

Please provide your step-by-step reasoning. User: While shopping for music online, Zoe bought 3 country albums and 5 pop albums. Each album came with a lyrics sheet and had 3 songs. How many songs did Zoe buy in total?
AI: Let us think step by step Let's think s

In [245]:
print(response_reason)

 Let us think step by step. The juggler can juggle 16 balls. Half of the balls are golf balls, so there are 8 golf balls. Half of the 8 golf balls are blue, so there are 4 blue golf balls. The answer is 4 blue golf balls.


<b><font color= 'green'>&#10004;</font> This is indeed the correct answer !

#### Trying one last math question to contrast chain of reasoning and non-chain of reasoning

In [246]:
question3='A pet store has 64 puppies. In one day, they sold 28 of them and put the rest into cages with 4 in each cage. How many cages did they use?'

#### Without chain of reason prompt

In [247]:
response_math3=llmchain_math.run(question3)
print(response_math3)



> Entering new  chain...
Prompt after formatting:
The following are excerpts from conversations with an AI math coach. The assistant provides answers to the users' questions. Here are some examples: 


User: I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?
AI: 10 apples



User: While shopping for music online, Zoe bought 3 country albums and 5 pop albums. Each album came with a lyrics sheet and had 3 songs. How many songs did Zoe buy in total?
AI: 24 songs


user:A pet store has 64 puppies. In one day, they sold 28 of them and put the rest into cages with 4 in each cage. How many cages did they use?
AI:

> Finished chain.
 14 cages


<b><font color= 'red'>&#10006;</font></b> This the wrong answer. The correct answer is (64-28)/4=9 cages

#### With chain of reason prompt

In [248]:
response_reason3=llmchain_reason.run(question3)
print(response_reason3)



> Entering new  chain...
Prompt after formatting:
The following are excerpts from conversations with an AI math coach. The assistant provides answers to the users' questions. Here are some examples: 

Please provide your step-by-step reasoning. User: I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?
AI: Let us think step by step Let's think step by step.First, you started with 10 apples. You gave away 2 apples to the neighbor and 2 to the repairman, so you had 6 apples left.
    Then you bought 5 more apples, so now you had 11 apples. Finally, you ate 1 apple, so you would remain with 10 apples.

Please provide your step-by-step reasoning. User: While shopping for music online, Zoe bought 3 country albums and 5 pop albums. Each album came with a lyrics sheet and had 3 songs. How many songs did Zoe buy in total?
AI: Let us think step by step Let's think s

<b><font color= 'green'>&#10004;</font> This is indeed the correct answer !